# About: コンテナイメージを更新する -- Moodle
---
構築環境のMoodleのコンテナイメージを更新する。

## 概要

Moodleのコンテナイメージを更新します。

![処理の流れ](images/moodle-061-01.png)

設定手順の主な流れを以下に示します。

1. Moodleのコンテナを停止する
1. 更新後のコンテナイメージから設定ファイル、PHPファイルを取得する
1. `docker-compose.yml`に記述したコンテナイメージの指定を更新する
1. Moodleコンテナを起動する

**このNotebookを実行する前に「811-AmazonS3へのバックアップ.ipynb」「812-scpによるバックアップ.ipynb」などを実行してバックアップを行なうことを推奨します**。

このNotebookではMoodleの[マイナーリリース](https://docs.moodle.org/dev/Process#Stable_maintenance_cycles)の更新を行うことを想定しています。メジャーリリースが異なるコンテナイメージへの更新は対象外となります。

## 準備

### グループ名の指定

このNotebookの操作対象となるAnsibleのグループ名を設定します。

既存のグループ名を確認するために`group_vars`にあるディレクトリの一覧を表示します。

In [ ]:
!ls -1 group_vars/

上のセルの出力結果を参考にしてAnsibleのグループ名を次のセルに指定してください。

In [ ]:
# (例)
# target_group = 'Moodle'

target_group = 

### 前提条件の確認

このNotebookを実行するための前提条件を満たしていることを確認します。

前提となる条件を以下に示します。

* 構築した際のパラメータが group_vars ファイルに記録されていること
* Ansibleから操作可能であること
* Ansibleから管理者権限でコマンドを実行できること

`target_group` に対応する設定ファイルが存在していることを確認します。

In [ ]:
from pathlib import Path

target_dir = Path('group_vars') / target_group
if not (target_dir.is_dir() and len(list(target_dir.glob("*"))) > 0):
    target_file = Path('group_vars') / f"{target_group}.yml"
    if target_file.is_file():
        target_dir.mkdir(exist_ok=True)
        target_file.rename(target_dir / "00-moodle.yml")
    else:
        raise RuntimeError(f"ERROR: not exists {target_group}")

対象となるホストにAnsibleで到達可能であることを確認します。

In [ ]:
!ansible {target_group} -m ping

管理者権限でコマンド実行可能であることを確認します。

In [ ]:
# 管理者権限(-b)でのコマンド実行
!ansible {target_group} -b -a 'whoami'

## パラメータ設定

### Moodleのコンテナイメージ

Moodleコンテナのイメージ名を指定してください。

現在設定されているコンテナイメージの値を確認します。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(target_group)
print(gvars["moodle_image_name"])

次のセルを実行すると選択可能なコンテナイメージの一覧を表示します。

> 次のセルの出力結果では存在している全てのコンテナイメージが表示されますが、このNotebookの設定手順ではメジャーバージョンが異なるコンテナイメージを指定することを想定していません。また現在利用しているバージョンより古いものを指定するとMoodleの起動はエラーとなります。

In [ ]:
import requests
import re
%run scripts/utils.py

url = f'https://harbor.vcloud.nii.ac.jp/api/repositories/vcp/moodle-simple/tags'
res = requests.get(url)
for x in sorted([
            f"harbor.vcloud.nii.ac.jp/vcp/moodle-simple:{x['name']}"
            for x in res.json()
            if not x['name'].endswith('-ssl') and re.match(r'^\d', x['name']) and check_version(x['name'])
        ], reverse=True):
    print(x)

次のセルに更新後のコンテナイメージを指定してください。

In [ ]:
# (例)
# moodle_image_name = 'harbor.vcloud.nii.ac.jp/vcp/moodle-simple:4.1.0'
# moodle_image_name = 'harbor.vcloud.nii.ac.jp/vcp/moodle-simple:4.1'

moodle_image_name = 

#### パラメータの保存

この節で指定したパラメータの値をファイルに保存します。

In [ ]:
%run scripts/group.py
update_group_vars(
    target_group,
    _file="20-moodle.yml",
    moodle_image_name=moodle_image_name,
)

`group_vars`ファイルの内容を確認してみます。

In [ ]:
!cat group_vars/{target_group}/20-moodle.yml

## メンテナンスモードへの切り替え

コンテナを停止するので、Moodleをメンテナンスモードに切り替えます。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle docker compose exec -T moodle \
    /usr/bin/php /var/www/html/admin/cli/maintenance.php --enable'

## 設定ファイルの配置

新しいコンテナイメージから設定ファイルをコピーしてホスト環境に配置します。

### コンテナの停止

現在実行中のMoodleコンテナを停止します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker compose stop moodle'

### 設定ファイルを取得する

指定されたコンテナイメージを取得し、そこからMoodleの設定ファイルとPHPファイルを取得します。

コンテナイメージから新しい設定ファイル、PHPファイルを取得する前に、`/srv/moodle/moodle/conf`, `/srv/moodle/moodle/data/php`に格納されている現在のファイルは`/srv/moodle/moodle/conf.{日時}`, `/srv/moodle/moodle/data/php.{日時}` に退避されます。

取得前後でのPHPファイルの差分を確認するために現在のPHPファイルのリストを記録しておきます。

In [ ]:
from datetime import datetime as dt
now = dt.now().strftime("%Y%m%d%H%M%S")
!ansible {target_group} -a 'chdir=/srv/moodle \
    mkdir -p tmp/{now}'
!ansible {target_group} -m shell -a 'chdir=/srv/moodle \
    find moodle/data/php -type f | sort > tmp/{now}/list.0'

実際に設定ファイル取得の操作を行う前にチェックモードで ansible を実行します。

In [ ]:
!ansible-playbook -l {target_group} -CDv playbooks/deploy-moodle-conf.yml || true

実際に設定ファイルの取得を行います。

In [ ]:
!ansible-playbook -l {target_group} -v playbooks/deploy-moodle-conf.yml

取得後の状態を確認します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    tree -L 2 moodle'

### PHPファイルの確認

このNotebookの更新手順ではMoodleが直接配布しているPHPファイルのみを対象としています。独自にインストールしたプラグインなどのファイルについては個別の対応が必要となります。ここでは更新前と更新後のPHPファイルの差分を示すことで、個別対応が必要となるプラグインなどに関する補足情報を示します。

新しく取得したPHPファイルのリストを作成します。

In [ ]:
!ansible {target_group} -m shell -a 'chdir=/srv/moodle \
    find moodle/data/php -type f | sort > tmp/{now}/list.1'

コンテナイメージを更新した前後でのPHPファイルリストの差分を確認します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    diff -u tmp/{now}/list.0 tmp/{now}/list.1' || true

更新前後のファイルの差異を参考にプラグインの更新などの対応を行なってください。差分が表示された場合でも個別にプラグインのインストールを行なっていない場合、対応は不要です。

## コンテナイメージの更新

moodleコンテナのイメージ指定を更新した`docker-compose.yml`を配置して、新しいコンテナの起動を行います。

### docker-compose.yml の配置

コンテナイメージを更新した`docker-compose.yml`を配置します。

In [ ]:
!ansible {target_group} -m template \
    -a 'src=template/docker-compose.yml.j2 \
        dest=/srv/moodle/docker-compose.yml backup=yes'

### コンテナ設定の変更を反映する

更新した設定でコンテナを起動します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker compose up -d --remove-orphans'

変更後のコンテナの状態を確認してみます。`State` の値が `Up` となっていることを確認してください。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker compose ps'

`State`が`Up`となっていない場合はコンテナのログなどを確認してください。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker compose logs --tail=50 moodle'

## メンテナンスモードの解除

設定変更が完了したのでメンテナンスモードを解除します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle docker compose exec -T moodle \
    /usr/bin/php /var/www/html/admin/cli/maintenance.php --disable'

## Moodle を利用できることを確認

Moodle環境にアクセスして利用可能であることを確認します。

次のセルを実行するとMoodleのアドレスを表示します。表示されたリンクをクリックしてMoodleにアクセスできることを確認してください。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(target_group)
print(gvars["moodle_url"])

Moodleにアクセスすると新しいバージョンへのアップグレードの処理が開始されることがあります。示された手順に従いMoodleの更新を行なってください。